In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Parameters
N_y, N_x = 9, 9  # grid size
l = 9                 # bar length (in arbitrary units)
r = 1.3              
dpi = 1000                # dots per inch for rendering

# Generate random angles for demonstration
A = np.random.uniform(-np.pi/2, np.pi/2, (N_y, N_x))
# Generate random contrasts for demonstration
W = np.random.uniform(0.5, 1, (N_y, N_x))
# Generate random saliency values for demonstration (replace with your own)
S = np.random.uniform(0.5, 2, (N_y, N_x))  # linewidths between 1 and 8

# Calculate image size in pixels
d = l * r # grid spacing
img_height = int(N_y * d)
img_width = int(N_x * d)
img_size = (img_height, img_width)

# Create figure
fig, ax = plt.subplots(figsize=(img_width/100, img_height/100), dpi=dpi)
ax.set_xlim(0, img_width)
ax.set_ylim(0, img_height)
ax.set_aspect('equal') # keep x and y scales the same, avoding distortion
ax.axis('off')

# Draw bars
for i in range(N_y):
    for j in range(N_x):
        # compute center of the bar
        cx = (j + 0.5) * d
        cy = (i + 0.5) * d
        # compute bar directions
        angle = A[i, j]
        dx = l * np.sin(angle) / 2
        dy = l * np.cos(angle) / 2
        # compute endpoints of the bar
        x0, y0 = cx - dx, cy - dy
        x1, y1 = cx + dx, cy + dy
        # draw the bar
        ax.plot([x0, x1], [y0, y1], 
                alpha=W[i, j], 
                color = "k", 
                linewidth=S[i, j], 
                solid_capstyle='butt'
        )

plt.gca().invert_yaxis()  # Optional: origin at top-left
plt.show()

In [ ]:
def plot_bars(A, W, l=9, r=1.3, verbose=True, dpi=500):
    """
    Plots a grid of bars with given angles, contrasts, and saliency (linewidths).
    
    Parameters:
        A (np.ndarray): 2D array of angles (radians), shape (N_y, N_x)
        W (np.ndarray or None): 2D array of linewidths, same shape as A
        l (float): Bar length
        r (float): Grid spacing factor
        verbose (bool): If True, show the plot
        dpi (int): Dots per inch for rendering
        
    Returns:
        fig (matplotlib.figure.Figure): The matplotlib figure object
    """
    assert A.ndim == 2, "A must be a 2D array"
    assert W.shape == A.shape, "C must have the same shape as A"
    N_y, N_x = A.shape
    
    # Calculate image size in pixels
    d = l * r # grid spacing
    img_height = int(N_y * d)
    img_width = int(N_x * d)

    # Create figure
    fig, ax = plt.subplots(figsize=(img_width/100, img_height/100), dpi=dpi)
    ax.set_xlim(0, img_width)
    ax.set_ylim(0, img_height)
    ax.set_aspect('equal') # keep x and y scales the same, avoding distortion
    ax.axis('off')

    # Draw bars
    for i in range(N_y):
        for j in range(N_x):
            # compute center of the bar
            cx = (j + 0.5) * d
            cy = (i + 0.5) * d
            # compute bar directions
            angle = A[i, j]
            dx = l * np.sin(angle) / 2
            dy = l * np.cos(angle) / 2
            # compute endpoints of the bar
            x0, y0 = cx - dx, cy - dy
            x1, y1 = cx + dx, cy + dy
            # draw the bar
            ax.plot([x0, x1], [y0, y1], 
                    color = "k", 
                    linewidth=W[i, j], 
                    solid_capstyle='butt'
            )
    
    if verbose:
        plt.show()
    
    return fig

def visualize_input(A, C, l=9, r=1.3, verbose=True, dpi=500):
    """ 
        Visualizes the input angles A and contrasts C as a grid of bars.
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians), shape (N_y, N_x)
            C (np.ndarray): 2D array of contrasts, same shape as A, values in [1, 4]
            l (float): Bar length
            r (float): Grid spacing factor
            verbose (bool): If True, show the plot
            dpi (int): Dots per inch for rendering
        
        Returns:
            fig (matplotlib.figure.Figure): The matplotlib figure object
    """
    
    assert np.all((C >= 1) & (C <= 4) | (C == 0)), "C values must 0 or in [1, 4]"
    W = C / 3
    return plot_bars(A, W, l=l, r=r, verbose=verbose, dpi=dpi)

def visualize_output(A, S, l=9, r=1.3, verbose=True, dpi=500):
    """ 
        Visualizes the output saliency S as a grid of bars with uniform orientation.
        
        Parameters:
            A (np.ndarray): 2D array of angles (radians), shape (N_y, N_x)
            S (np.ndarray): 3D array (Y x X) of saliency values, shape (N_y, N_x)
            l (float): Bar length
            r (float): Grid spacing factor
            verbose (bool): If True, show the plot
            dpi (int): Dots per inch for rendering
        
        Returns:
            fig (matplotlib.figure.Figure): The matplotlib figure object
    """
    # TODO: how to scale and normalize when reading out S?
    assert np.all(S >= 0), "S values must be non-negative"
    return plot_bars(A, S, l=l, r=r, verbose=verbose, dpi=dpi)

In [ ]:
# test_main_naive.ipynb

import matplotlib.figure

def test_bar_without_surround(verbose=False):
    C = np.zeros((9, 9)) 
    C[4, 4] = 3.5
    A = np.zeros((9, 9))
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

def test_iso_orientation(verbose=False):
    C = np.full((9, 9), 3.5)
    A = np.zeros((9, 9))
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

def test_random_background(verbose=False, seed=42):
    C = np.full((9, 9), 3.5)
    rng = np.random.default_rng(seed)
    A = rng.uniform(-np.pi/2, np.pi/2, (9, 9))
    A[4, 4] = 0.
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

def test_cross_orientation(verbose=False):
    C = np.full((9, 9), 3.5)
    A = np.full((9, 9),  np.pi / 2)
    A[4, 4] = 0
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)
    
def test_bar_without_surround_low_contrast(verbose=False):
    C = np.zeros((9, 9))
    C[4, 4] = 1.05
    A = np.zeros((9, 9))
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

def test_with_one_flanker(verbose=False):
    C = np.zeros((9, 9))
    C[4, 4] = 1.05
    C[5, 4] = 3.5
    A = np.zeros((9, 9))
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

def test_with_two_flankers(verbose=False):
    C = np.zeros((9, 9))
    C[4, 4] = 1.5
    C[3, 4] = 3.5
    C[5, 4] = 3.5
    A = np.zeros((9, 9))
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

def test_with_flanking_line_and_noise(verbose=False):
    rng = np.random.default_rng(42)
    A = rng.uniform(-np.pi/2, np.pi/2, (9, 9))
    A[:, 4] = 0.
    C = np.full((9, 9), 3.5)
    C[4, 4] = 1.5
    fig = visualize_input(A, C, verbose=verbose)
    assert isinstance(fig, matplotlib.figure.Figure)

# run tests
test_bar_without_surround(verbose=True)
test_iso_orientation(verbose=True)
test_random_background(verbose=True)
test_cross_orientation(verbose=True)

test_bar_without_surround_low_contrast(verbose=True)
test_with_one_flanker(verbose=True)
test_with_two_flankers(verbose=True)
test_with_flanking_line_and_noise(verbose=True)
